## Google Sentence Piece를 이용해서 Vocab 파일을 만드는 과정
Google SentencePeice와 네이버 리뷰 데이터를 이용해서 Vocab을 만드는 과정에 대한 설명 입니다.

[Colab](https://colab.research.google.com/)에서 실행 했습니다.

#### 0. Pip Install
필요한 패키지를 pip를 이용해서 설치합니다.

In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.6 MB/s eta 0:00:0000:0100:01m


#### 1. Google Drive Mount
Colab에서는 컴퓨터에 자원에 접근이 불가능 하므로 Google Drive에 파일을 올려 놓은 후 Google Drive를 mount 해서 로컬 디스크처럼 사용 합니다.
1. 아래 블럭을 실행하면 나타나는 링크를 클릭하세요.
2. Google 계정을 선택 하시고 허용을 누르면 나타나는 코드를 복사하여 아래 박스에 입력한 후 Enter 키를 입력하면 됩니다.

학습관련 [데이터 및 결과 파일](https://drive.google.com/open?id=15XGr-L-W6DSoR5TbniPMJASPsA0IDTiN)을 참고 하세요.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# data를 저장할 폴더 입니다. 환경에 맞게 수정 하세요.
data_dir = "data/"

ModuleNotFoundError: No module named 'google.colab'

#### 2. Imports


In [ ]:
import os
import gzip
import pandas as pd
import sentencepiece as spm
import shutil

#### 3. 폴더의 목록을 확인
Google Drive mount가 잘 되었는지 확인하기 위해 data_dir 목록을 확인 합니다.

In [ ]:
for f in os.listdir(data_dir):
  print(f)

#### 4. CSV 파일을 TEXT 형태로 변환
Google SentencePiece에서 사용할 수 있도록 CSV를 TEXT형태로 변환 합니다.

In [ ]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
# 두 텍스트 파일을 열어서 내용을 읽습니다.
with open('ratings_train.txt', 'r') as file1:
    content1 = file1.read()

with open('ratings_test.txt', 'r') as file2:
    content2 = file2.read()

# 두 파일의 내용을 합쳐서 새로운 파일에 저장합니다.
with open('merged_file.txt', 'w') as merged_file:
    merged_file.write(content1 + "\n" + content2)


#### 5. Vocab 생성
Google SentencePiece를 이용해서 vocab 만드는 방법 입니다. vocab을 만드는데 많은 시간이 소요 됩니다. 잠시 여유를 갖고 기다려 주세요.

In [ ]:
corpus = "merged_file.txt"
prefix = "naver_review"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 기타 추가 토큰

for f in os.listdir("."):
  print(f)
shutil.copy("naver_review.model", f"{data_dir}/naver_review.model")
shutil.copy("naver_review.vocab", f"{data_dir}/naver_review.vocab")

#### 6. Vocab 테스트
생성된 vocab을 간단하게 테스트 해 봅니다.

In [ ]:
vocab_file = f"{data_dir}/naver_review.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()